In [23]:
import alibi_detect
alibi_detect.od.__all__
from pathlib import Path
import numpy as np
import pandas as pd
from alibi_detect.saving import save_detector, load_detector


In [24]:
from sklearn.model_selection import train_test_split
data = pd.read_csv("..\\data\\urls_with_features_selected.csv")
api_data = pd.read_csv("..\\data\\api_urls.csv")
X = data.loc[:, data.columns != 'type']
y = data['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
import os, sys
from alibi_detect.od import IForest
from pathlib import Path 

detector_name = 'IForest'

od = IForest(threshold=0.5,  # threshold for outlier score
                 n_estimators=100)
    
    # train
od.fit(X_train)
    
    # save the trained outlier detector
save_detector(od, "\\outlier_detector")

In [26]:
od.fit(X_train)

In [27]:
od.infer_threshold(
    X,
    threshold_perc=95
)

In [30]:
preds = od.predict(
    api_data.loc[:, data.columns != 'type'],
    return_instance_score=True
)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- numAmpersands
- numSubdomains
Feature names seen at fit time, yet now missing:
- numApersand
- subdomainLevel


In [ ]:
preds

{'data': {'instance_score': array([-0.15018496, -0.14425049, -0.14108286, ...,  0.07992756,
         -0.14678026, -0.05864491]),
  'feature_score': None,
  'is_outlier': array([0, 0, 0, ..., 1, 0, 0])},
 'meta': {'name': 'IForest',
  'online': False,
  'data_type': 'tabular',
  'version': '0.11.5dev',
  'detector_type': 'outlier'}}